In [1]:
# Google Colab: count leaflets per year + per (year, manual_topic)
# Assumes you upload a CSV with columns: id, manual_topic, year

import pandas as pd
import numpy as np
from google.colab import files

# 1) Upload CSV
uploaded = files.upload()
csv_path = next(iter(uploaded.keys()))
print("Loaded:", csv_path)

# 2) Read + clean
df = pd.read_csv(csv_path)

# normalize column names just in case
df.columns = [c.strip().lower() for c in df.columns]
required = {"id", "manual_topic", "year"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Missing columns in CSV: {missing}. Found: {list(df.columns)}")

df["id"] = df["id"].astype(str).str.strip()
df["manual_topic"] = df["manual_topic"].astype(str).str.strip()
df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")

# optional: unify e.g. "Foreign/War " -> "Foreign/War"
df["manual_topic"] = df["manual_topic"].str.replace(r"\s+", " ", regex=True)

# 3) Keep only unique leaflets: keep "a" when there is a/b, otherwise keep the one without suffix
# Rule:
# - base_id = digits at start
# - suffix = trailing letters (a/b/...)
# - if group has suffixes, keep 'a' if present; else keep first row
import re

id_pat = re.compile(r"^(?P<base>\d+)(?P<suf>[A-Za-z]*)$")

tmp = df["id"].str.extract(id_pat)
df["base_id"] = tmp["base"]
df["suffix"] = tmp["suf"].str.lower().fillna("")

def pick_one(group: pd.DataFrame) -> pd.DataFrame:
    # prefer suffix == 'a' if it exists
    a = group[group["suffix"] == "a"]
    if len(a) > 0:
        return a.iloc[[0]]
    # otherwise prefer empty suffix (no letter) if exists
    no_suf = group[group["suffix"] == ""]
    if len(no_suf) > 0:
        return no_suf.iloc[[0]]
    # else just take first
    return group.iloc[[0]]

df_u = (
    df.groupby("base_id", as_index=False, group_keys=False)
      .apply(pick_one)
      .reset_index(drop=True)
)

print("Rows in CSV:", len(df))
print("Unique leaflets kept:", len(df_u))

# 4) Counts per year (total)
counts_year = (
    df_u.groupby("year")
        .size()
        .rename("total")
        .reset_index()
        .sort_values("year")
)
display(counts_year)

# 5) Counts per year x topic (pivot table)
pivot = (
    df_u.pivot_table(index="year", columns="manual_topic", values="id",
                     aggfunc="count", fill_value=0)
        .sort_index()
)

# add total column
pivot["Kopā"] = pivot.sum(axis=1)

display(pivot)

# 6) Export to Excel (easy to paste into PowerPoint / keep as file)
out_xlsx = "counts_by_year_and_topic.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as w:
    counts_year.to_excel(w, index=False, sheet_name="per_year_total")
    pivot.reset_index().to_excel(w, index=False, sheet_name="per_year_topic")
print("Saved:", out_xlsx)

files.download(out_xlsx)


Saving manual_annotation_topics_1_column.csv to manual_annotation_topics_1_column.csv
Loaded: manual_annotation_topics_1_column.csv
Rows in CSV: 266
Unique leaflets kept: 251


/tmp/ipython-input-3979660031.py:57: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(pick_one)


,year,total
0,1934,94
1,1935,72
2,1936,29
3,1937,15
4,1938,13
5,1939,21
6,1940,7


manual_topic,Aid,Foreign/War,Labour,Other,Spain,Women,Youth,Kopā
year,,,,,,,,
1934,14,15,37,8,0,2,18,94
1935,19,12,25,4,0,4,8,72
1936,4,5,6,4,5,0,5,29
1937,0,6,5,0,1,0,3,15
1938,0,8,2,1,0,0,2,13
1939,0,13,6,0,0,0,2,21
1940,0,2,4,0,0,0,1,7


Saved: counts_by_year_and_topic.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>